In [ ]:
from openai import OpenAI

API_KEY = "API_KEYS"
BASE_URL = "https://api.sree.shop/v1"

client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

In [2]:
try:
    models = client.models.list().data
    for model in models:
        print(model.id)
except Exception as e:
    print("Error listing models:", e)

claude-3-5-sonnet-20240620
claude-3-5-sonnet
deepseek-ai/DeepSeek-R1-Distill-Qwen-32B
deepseek-r1
deepseek-v3
gpt-4o
gpt-4o-2024-05-13
Meta-Llama-3.3-70B-Instruct-Turbo


In [4]:
import pandas as pd

df = pd.read_csv('reviews.csv')

df.head()

,Date,reviews
0,2024-12-27T12:57:43Z,Mera sath bhi huya hai same
1,2024-12-27T12:35:28Z,Same experience we also suffered for 4 daysâ€¦...
2,2024-12-27T10:04:07Z,Mere sath bhee same story ho chuki hai
3,2024-12-26T19:00:32Z,Mam aapne apni nahi ham sabki aankhe khol di k...
4,2024-12-26T16:21:14Z,ðŸ˜¢ðŸ˜¢


In [5]:
missing_reviews = df[df['reviews'].isnull()]
print(missing_reviews)

                       Date reviews
1191   2024-12-23T10:13:40Z     NaN
8114   2024-12-06T10:42:28Z     NaN
10904  2024-10-09T08:35:20Z     NaN
13283  2024-11-12T13:16:42Z     NaN
15177  2024-12-01T18:16:37Z     NaN
16961  2024-11-09T13:47:57Z     NaN
17040  2024-11-05T11:24:24Z     NaN
17084  2024-11-05T11:27:44Z     NaN
17117  2024-11-05T11:29:48Z     NaN
17162  2024-11-05T11:31:18Z     NaN


In [6]:
missing_reviews = df[df['reviews'].isnull()]
print(missing_reviews)

                       Date reviews
1191   2024-12-23T10:13:40Z     NaN
8114   2024-12-06T10:42:28Z     NaN
10904  2024-10-09T08:35:20Z     NaN
13283  2024-11-12T13:16:42Z     NaN
15177  2024-12-01T18:16:37Z     NaN
16961  2024-11-09T13:47:57Z     NaN
17040  2024-11-05T11:24:24Z     NaN
17084  2024-11-05T11:27:44Z     NaN
17117  2024-11-05T11:29:48Z     NaN
17162  2024-11-05T11:31:18Z     NaN


In [7]:
# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

In [8]:
# Remove duplicates
df = df.drop_duplicates()

In [9]:
def translate_text_with_openai(text):
    try:
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "user", "content": f"just write translation of the text in english: {text}"}
            ]
        )
        return completion.choices[0].message.content
    except Exception as e:
        print("Error:", e)
        return None

# Example usage
text = "Mera sath bhi huya hai same"
translated_text = translate_text_with_openai(text)
print(translated_text)

"I have also experienced the same."


In [11]:
import time

# Select the first 100 rows (use .copy() to avoid SettingWithCopyWarning)
df_first_100 = df.head(100).copy()

def process_review(row):
    translated_text = translate_text_with_openai(row['reviews'])
    return pd.Series([translated_text])

# Apply the function to the dataframe with rate limiting
for i in range(0, len(df_first_100), 3):
    batch = df_first_100.iloc[i:i+3]
    # Extract the resulting first column from the apply result before setting
    df_first_100.loc[batch.index, 'translated_text'] = batch.apply(process_review, axis=1)[0]
    time.sleep(60)  # Sleep for 60 seconds to avoid rate limiting

# Print the result
print(df_first_100)

Error: Error code: 429 - {'error': 'Rate limit exceeded - 3 requests per 60s', 'status': 429}
Error: Error code: 429 - {'error': 'Rate limit exceeded - 3 requests per 60s', 'status': 429}
Error: Error code: 429 - {'error': 'Rate limit exceeded - 3 requests per 60s', 'status': 429}
Error: Error code: 429 - {'error': 'Rate limit exceeded - 3 requests per 60s', 'status': 429}
Error: Error code: 429 - {'error': 'Rate limit exceeded - 3 requests per 60s', 'status': 429}
Error: Error code: 429 - {'error': 'Rate limit exceeded - 3 requests per 60s', 'status': 429}
Error: Error code: 429 - {'error': 'Rate limit exceeded - 3 requests per 60s', 'status': 429}
                        Date  \
0  2024-12-27 12:57:43+00:00   
1  2024-12-27 12:35:28+00:00   
2  2024-12-27 10:04:07+00:00   
3  2024-12-26 19:00:32+00:00   
4  2024-12-26 16:21:14+00:00   
..                       ...   
95 2024-12-16 19:23:14+00:00   
96 2024-12-16 18:34:37+00:00   
97 2024-12-16 17:50:10+00:00   
98 2024-12-16 16:08:25

In [14]:
df_first_100.iloc[0:100]

,Date,reviews,translated_text
0,2024-12-27 12:57:43+00:00,Mera sath bhi huya hai same,None
1,2024-12-27 12:35:28+00:00,Same experience we also suffered for 4 daysâ€¦...,None
2,2024-12-27 10:04:07+00:00,Mere sath bhee same story ho chuki hai,None
3,2024-12-26 19:00:32+00:00,Mam aapne apni nahi ham sabki aankhe khol di k...,"""Mom, you have opened not just your eyes but a..."
4,2024-12-26 16:21:14+00:00,ðŸ˜¢ðŸ˜¢,Request error occurred:
...,...,...,...
95,2024-12-16 19:23:14+00:00,Aaj mere papa ji k sath bhi esa hi hua but hmn...,"""Today something similar happened with my dad,..."
96,2024-12-16 18:34:37+00:00,Thanks for alerting us.,Thanks for alerting us.
97,2024-12-16 17:50:10+00:00,Maine bhi kiya aj15000,"""I also did it today for 15,000."""
98,2024-12-16 16:08:25+00:00,Same aisa mere sath bhi hua hai iske khilaf go...,"""Something similar has happened to me as well;..."


In [15]:
# download this dataframe as a csv file
df_first_100.to_csv('translated_reviews.csv', index=False)